# Hack to fix errors in the SQL Alchemy python library  

> A very lightweight script to be run on `sqla` code that has been freshly generated from LinkML. This adds polymorphism to children of InformationContentEntity classes and adds primaryjoin/secondaryjoin to self-referential relationships.

In [ ]:
#| default_exp hacks.sqla_fixes

In [ ]:
#| export

import os
import re
import local_resources.linkml as linkml
from importlib_resources import files

# Edits to SQL Alchemy Source Code

In [ ]:
#| export

sqla_source_code = files(linkml).joinpath('schema_sqla.py').read_text()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gburns/Documents/Coding/ChatGPT_etc/alzhazen/local_resources/linkml/schema_sqla.py'

In [ ]:
#| export

source = '''has_notes = relationship\( "Note", secondary="Note_has_notes"\)'''
target = '''has_notes = relationship( "Note", 
                             secondary="Note_has_notes", 
                             primaryjoin="Note.id==Note_has_notes.c.Note_id",
                             secondaryjoin="Note.id==Note_has_notes.c.has_notes_id",
                             back_populates="is_about")'''
if re.search(source, sqla_source_code):
    sqla_source_code = re.sub(source, target, sqla_source_code)

In [ ]:
#| export

source = '''(return f"InformationContentEntity\\(creation_date={.*\n+\s+#.*\n\s+__mapper_args__ = )(\{\n.*?\n\s+\})\n'''
target = '''\g<1>{
        'concrete': True,
        'polymorphic_on': type,
        "polymorphic_identity": "InformationContentEntity",
    }'''
if re.search(source, sqla_source_code, re.MULTILINE):
    sqla_source_code = re.sub(source, target, sqla_source_code, re.MULTILINE)

In [ ]:
#| export

source = '''(return f"<<<SUBHERE>>>\\(.*\n+\s+#.*\n\s+__mapper_args__ = )(\{\n.*?\n\s+\})\n'''
target = '''\g<1>{
        'concrete': True,
        "polymorphic_identity": "<<<SUBHERE>>>",
    }'''
classes_to_edit = ['Note', 
                   'Author', 
                   'Organization', 
                   'ScientificKnowledgeCollection', 
                   'ScientificKnowledgeExpression', 
                   'ScientificKnowledgeFragment',
                   'ScientificKnowledgeItem']
for c in classes_to_edit:
    s = source.replace('<<<SUBHERE>>>', c)
    t = target.replace('<<<SUBHERE>>>', c)
    if re.search(s, sqla_source_code, re.MULTILINE):
        sqla_source_code = re.sub(s, t, sqla_source_code, re.MULTILINE)

In [ ]:
#| export

source = '''has_notes = relationship\( "Note", secondary="<<<SUBHERE>>>_has_notes"\)\n'''
target = '''has_notes = relationship( "Note", secondary="<<<SUBHERE>>>_has_notes", back_populates="is_about")\n'''
classes_to_edit = ['InformationContentEntity',
                   'Note', 
                   'Author', 
                   'Organization', 
                   'ScientificKnowledgeCollection', 
                   'ScientificKnowledgeExpression', 
                   'ScientificKnowledgeFragment',
                   'ScientificKnowledgeItem']
for c in classes_to_edit:
    s = source.replace('<<<SUBHERE>>>', c)
    t = target.replace('<<<SUBHERE>>>', c)
    if re.search(s, sqla_source_code, re.MULTILINE):
        sqla_source_code = re.sub(s, t, sqla_source_code, re.MULTILINE)

In [ ]:
#| export

source = '''is_about = relationship\( "InformationContentEntity", secondary="Note_is_about"\)'''
target = '''is_about = relationship( "InformationContentEntity", secondary="Note_is_about", back_populates="has_notes")'''
if re.search(source, sqla_source_code, re.MULTILINE):
    sqla_source_code = re.sub(source, target, sqla_source_code, re.MULTILINE)

In [ ]:
#| export

os.rename(files(linkml).joinpath('schema_sqla.py'), files(linkml).joinpath('schema_sqla_original.py'))
files(linkml).joinpath('schema_sqla.py').write_text(sqla_source_code)

39684

# Edits to Raw SQL Code

In [ ]:
#| export
import difflib

# Delete all foreign keys
raw_sql_source_code = files(linkml).joinpath('schema.sql').read_text()
lines = [l for l in raw_sql_source_code.splitlines() if 'FOREIGN' not in l]
lines = [re.sub('\),\s*$', ')', l) if 'PRIMARY' in l else l for l in lines ]
raw_sql_source_code = '\n'.join(lines)

print(raw_sql_source_code)


CREATE TYPE "ScientificKnowledgeExpressionType" AS ENUM ('ScientificPrimaryResearchArticle', 'ScientificPrimaryResearchPreprint', 'ScientificReviewArticle', 'ScientificBook', 'ScientificBookChapter', 'ScientificConferenceArticle', 'ScientificDissertation');CREATE TYPE "NoteType" AS ENUM ('NoteAboutCollection', 'NoteAboutExpression', 'NoteAboutFragment');
CREATE TABLE "Entity" (
	id TEXT NOT NULL, 
	type TEXT NOT NULL, 
	PRIMARY KEY (id)
);COMMENT ON TABLE "Entity" IS 'Root Model class for all things and informational relationships, real or imagined.';COMMENT ON COLUMN "Entity".id IS 'A simple, locally-generated unique identifier specific with different heuristic formatting for each application.';COMMENT ON COLUMN "Entity".type IS 'The type of an Entity expressed as curi.';
CREATE TABLE "NamedThing" (
	name TEXT, 
	id TEXT NOT NULL, 
	type TEXT NOT NULL, 
	PRIMARY KEY (id)
);COMMENT ON TABLE "NamedThing" IS 'an entity or concept/class described by a name';COMMENT ON COLUMN "NamedThing".

In [ ]:
#| export

os.rename(files(linkml).joinpath('schema.sql'), files(linkml).joinpath('schema_original.sql'))
files(linkml).joinpath('schema.sql').write_text(raw_sql_source_code)


39476